# Temporally Coherent and Spatially Accurate Video Matting
  
虽然这篇文章不是DL做的，但是也是很有参考价值（我瞎猜的，主要Deep automatic portrait image matting引用有这篇文章，看起来还不错准备瞅瞅）

这篇文章我最关注的点，是多种传统图像处理方法之间怎么结合的，虽然我现在用了canny+dilate+closed-form，取得了一定的效果，但是对与pixel-based algorithm来说还有很多事情可以做

## 1. 方法介绍
  
这里直接跳到论文第二部分propesed method，本文方法和其他sampling-based方法架构类似。  
首先收集一组前景和背景的样本（即子区域），然后选择最佳的$(F,B)$（即能使得后续算法更好处理的一组）。接着使用matting Laplacian方法来精化图层。

主要就是在选择最佳的$(F,B)$的目标函数设计中加了时序项（temporal term），而最后也是用了拉普拉斯矩阵做了精化提高精度。

<img src='./img/201.png' alt='Opps' align='center' width=600px>

可以清楚的从论文中的分析图中看出，本文算法是对颜色、纹理一次做了两层（次）聚类，具体怎么做我们往下看。

说句题外话，其实从这个图上我们能直观得得出图像分割的几个步骤：
  
1. 确定兴趣物体（即你要分割哪个/些物体）
2. 定位物体边界（内边界和外边界）
3. 精化/确定位物体边界
  
确定兴趣物体很好理解，我们的分割/抠图，都是有目的的，那么这个目标物体就是我们的目的。而第二点，定位物体边界，这里就涉及到多种方法了。早期的pixel-based的方法，基于RGB颜色空间最多加个位置信息，其实都是被图像存储格式所影响。实际上我们应该先确定物体结构，通过目标检测、边缘检测等方法，定位目标具体位置，这也是比较麻烦的一个步骤，当然这部分处理好了对接下来精确的边界定位起到非常大的影响。内边界和外边界的选取是为了接下来的步骤做准备，可以看下closed-form-matting等运用Laplacian Matrix求解前景背景区域的。这类算法都是假设在小区域内，前景和背景都是由两种颜色线性组合而成的，而且有公式$$I = \alpha F+(1-\alpha)B$$实际上到了小区域也就是求解这个方差中的$\alpha$值。  
  
  
  

## 2 局部边界以及时序全局采样

通过采样来抠图，结果的好坏很大程度上依赖选取的样本点（图像太大考虑全部像素不合实际），如果前景或背景中有像素没有被涵盖将很大程度上影响结果。  
为了解决这个问题，本文用了基于局部和时序的方法来提取样本点。首先是局部边界采样（local boundary sample），这部分的采样结果是一组高关联的样本点。（这里我连接高关联 highly correlated 实际上就是沿着边缘，所以关联性强），然后在时序上采集全局的样本点，来表征前景和背景在颜色及纹理上的变化。  

文中保证所有颜色都被采样表征的方法来源于文章:  
>S HAHRIAN E., R AJAN D., P RICE B., C OHEN S.: Improving image matting using comprehensive sampling sets. In CVPR (2013).  
  
